In [2]:
from sklearn.metrics import classification_report
from scipy.misc import imread
import pandas as pd
import numpy as np
import glob
import re

In [3]:
#Clean up user defined data in the weather column of the weather dataset

In [4]:
# flatten in imread set to true, images loaded as gray scale
row = 0
image_data = pd.DataFrame(columns=("image", "timestamp"))

for path in sorted(glob.glob("./katkam-scaled/*")):
    
    matches = re.search("\d+", path)
    image_data.loc[row] = [imread(path, flatten=True).flatten(), matches.group()]
    row += 1

image_data["timestamp"] = pd.to_datetime(image_data["timestamp"])

In [6]:
# get list of weather data csv files
files = [f for f in sorted(glob.glob("./yvr-weather/*"))]

In [7]:
import datetime
#images only provided from 6 am to 9 pm at night
def CleanWeatherData(weather_data):
    # data quality <-- look at later? 
    weather_data.drop(["Year","Month","Day","Time","Temp Flag", "Dew Point Temp Flag",
                       "Rel Hum Flag", "Wind Dir Flag", "Wind Spd Flag",
                       "Visibility Flag", "Stn Press Flag",
                       "Hmdx Flag", "Wind Chill Flag", "Hmdx", "Wind Chill",
                       "Data Quality"], axis=1, inplace=True)
    weather_data.rename(columns={'Date/Time':'timestamp'}, inplace=True)
    
filename = files[0]
weather_data = pd.read_csv(filename, sep=',', skiprows=16, parse_dates=[0])
weather_data2 = pd.read_csv(files[1], sep=',', skiprows=16, parse_dates=[0])
#weather_data3 = pd.read_csv(files[2], sep=',', skiprows=16, parse_dates=[0])

weather_data = weather_data.append(weather_data2)


CleanWeatherData(weather_data)
#CleanWeatherData(weather_data2)
#CleanWeatherData(weather_data3)

In [5]:
image_data["timestamp"].dtype

dtype('<M8[ns]')

In [8]:
# Clean up words descriptions Mostly, Mainly, Rain Showers, Heavy Rain, Drizzle <-?, 
def CleanDescription(weather):
    #re.sub("(Moderate[\s]+)*(Mostly[\s]+)*(Mainly[\s]+)*([\s]+Showers)*","", weather["Weather"])
    regex_string = "(Moderate[\s]+)*(Mostly[\s]+)*(Mainly[\s]+)*([\s]+Showers)*"
    weather["Weather"] = weather["Weather"].str.replace(regex_string, "")
    no_drizzle = "Drizzle"
    weather["Weather"] = weather["Weather"].str.replace(no_drizzle, "Rain")
    
    

#Takes the original weather data 
#Split the data into 
def dataFrame2Array(images, train_set):
    # reshape to (_,49152) (192x256 image to 1-d)
    
    for i in range(train_set.shape[0]):
        images[i] = train_set.image.iloc[i] / 255
    
    
def splitDataWithNanValues(images, weather):
    
    #Split into non null and null
    train = weather[~weather["Weather"].isnull()]
    test = weather[weather["Weather"].isnull()]

    train_times = list(train.timestamp.map(lambda x: x))
    test_times = list(test.timestamp.map(lambda x: x))
    
    img_train = images[images.timestamp.isin(train_times)]
    img_test = images[images.timestamp.isin(test_times)]
    
    train_img_times = list(img_train.timestamp.map(lambda x: x))
    test_img_times = list(img_test.timestamp.map(lambda x: x))
    
    y_train = train[train.timestamp.isin(train_img_times)]
    y_test = test[test.timestamp.isin(test_img_times)]
    
    X_train = np.zeros((img_train.shape[0],49152), dtype=np.float32)
    X_test = np.zeros((img_test.shape[0],49152), dtype=np.float32)
    
    dataFrame2Array(X_train, img_train)
    dataFrame2Array(X_test, img_test)
    
    return X_train, X_test, y_train, y_test


In [9]:
CleanDescription(weather_data)
CleanDescription(weather_data2)
CleanDescription(weather_data3)


X_train, X_test, y_train, y_test = splitDataWithNanValues(image_data, weather_data)
X_train2, X_test2, y_train2, y_test2 = splitDataWithNanValues(image_data, weather_data2)
X_train3, X_test3, y_train3, y_test3 = splitDataWithNanValues(image_data, weather_data3)

X = np.concatenate((X_train,X_train3), axis=0)
y = pd.concat((y_train, y_train3), ignore_index=True)

In [11]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train.Weather)
y_predicted = model.predict(X_train2)
print(classification_report(y_train2.Weather,y_predicted))

             precision    recall  f1-score   support

      Clear       0.81      0.73      0.77        70
     Cloudy       0.67      0.68      0.68        72
       Rain       0.54      0.71      0.61        21
   Rain,Fog       0.00      0.00      0.00         1

avg / total       0.71      0.70      0.70       164



/home/seleena/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=50)
model.fit(X, y.Weather)
#print(model.score(X_train2, y_train2.Weather))
y_predicted = model.predict(X_train2)
print(classification_report(y_train2.Weather,y_predicted))

             precision    recall  f1-score   support

      Clear       0.87      0.66      0.75        70
     Cloudy       0.64      0.85      0.73        72
       Rain       0.69      0.52      0.59        21
   Rain,Fog       0.00      0.00      0.00         1

avg / total       0.74      0.72      0.72       164



/home/seleena/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Notes
- try black and white images
- try rgb images
- try other color coding images
- https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
- use temp, dew point, ... etc to estimate weather conditions

Try:
1. remove nan data -> use for testing
2. impute nan weather data -> compare to #1